# Analysis of TCGA RNASeq data using DESeq2
This notebooks uses DESeq2 and R2Py to analyze the RNASeq data sets from TCGA.

In [12]:
import os
import sys
import pandas as pd # version 0.19
import rpy2 # version 2.9.1
from rpy2.robjects import pandas2ri, Formula, r

Note that it is required to install the "DESeq2" package in R. For that run:


if (!requireNamespace("BiocManager", quietly = TRUE))

    install.packages("BiocManager")
    
BiocManager::install("DESeq2")

In [13]:
pandas2ri.activate()
from rpy2.robjects.packages import importr
deseq = importr('DESeq2')

In [14]:
rpy2.__version__

'2.9.1'

In [15]:
pd.__version__

'0.19.0'

Python class defined in https://gist.github.com/wckdouglas/3f8fb27a3d7a1eb24c598aa04f70fb25

In [16]:
to_dataframe = r('function(x) data.frame(x)')

class py_DESeq2:

    def __init__(self, count_matrix, design_matrix, design_formula, gene_column='id'):
        try:
            assert gene_column in count_matrix.columns, 'Wrong gene id column name'
            gene_id = count_matrix[gene_column]
        except AttributeError:
            sys.exit('Wrong Pandas dataframe?')

        self.dds = None
        self.deseq_result = None
        self.resLFC = None
        self.comparison = None
        self.normalized_count_matrix = None
        self.gene_column = gene_column
        self.gene_id = count_matrix[self.gene_column]
        
        count_matrix = count_matrix.drop(gene_column,axis=1)
        
        print(f'{count_matrix.shape[1]} | {design_matrix.shape[0]}')
        
        # Load dataframe into R environment
        # Important: Change to r.data() if you use numpys and rpy2 latests versions
        count_matrix = pandas2ri.py2ri(count_matrix)
        
        # Assign columns to NULL
        count_matrix.names = rpy2.rinterface.NULL
        
        self.count_matrix = count_matrix
        
        self.design_matrix = pandas2ri.py2ri(design_matrix)
        
        self.design_formula = Formula(design_formula)


    def run_deseq(self, **kwargs):
        self.dds = deseq.DESeqDataSetFromMatrix(
            countData=self.count_matrix, 
            colData=self.design_matrix,
            design=self.design_formula
        )
        self.dds = deseq.DESeq(self.dds, **kwargs)
        # Previous script had "deseq.counts" instead
        self.normalized_count_matrix = deseq.counts_DESeqDataSet(self.dds, normalized=True)

    def get_deseq_result(self, **kwargs):

        self.comparison = deseq.resultsNames(self.dds)

        self.deseq_result = deseq.results(self.dds, **kwargs)
        self.deseq_result = to_dataframe(self.deseq_result)
        self.deseq_result = pandas2ri.ri2py(self.deseq_result) ## back to pandas dataframe
        self.deseq_result[self.gene_column] = self.gene_id.values
        return self.deseq_result

Get and preprocessing data

In [17]:
# Point to the data folder
dir_path = os.path.dirname(os.path.realpath('__file__'))
SOURCE = os.path.join(os.path.abspath(os.path.join(dir_path, os.pardir)))
DATA = os.path.join(SOURCE, 'data', 'rnaseq')

lihc_counts_path = os.path.join(DATA,'lihc_read_count_duplicates_removed.txt')
lihc_labels_path = os.path.join(DATA,'LIHC_transposed_labels.txt')

In [18]:
lihc_count_matrix = pd.read_csv(
    lihc_counts_path, 
    sep = ',', 
)

lihc_design_matrix = pd.read_csv(
    lihc_labels_path,
    sep = ',',
    index_col=0
)

In [19]:
lihc_count_matrix.shape

(55150, 422)

In [20]:
lihc_count_matrix.head()

,gene_symbol,TCGA-DD-A1EE-11A-11R-A131-07,TCGA-EP-A26S-11A-12R-A16W-07,TCGA-DD-A114-11A-12R-A131-07,TCGA-BD-A3EP-11A-12R-A22L-07,TCGA-ES-A2HT-11A-11R-A180-07,TCGA-DD-A3A6-11A-11R-A22L-07,TCGA-FV-A3I1-11A-11R-A22L-07,TCGA-DD-A118-11A-11R-A131-07,TCGA-DD-A11A-11A-11R-A131-07,...,TCGA-DD-A3A1-01A-11R-A213-07,TCGA-DD-A1EH-01A-11R-A131-07,TCGA-CC-A7IL-01A-11R-A33R-07,TCGA-G3-A5SL-01A-11R-A27V-07,TCGA-BC-A5W4-01A-11R-A28V-07,TCGA-DD-A1EA-01A-11R-A131-07,TCGA-EP-A2KB-01A-11R-A180-07,TCGA-DD-AAW3-01A-11R-A41C-07,TCGA-BC-4073-01B-02R-A131-07,TCGA-RC-A6M6-01A-11R-A32O-07
0,TSPAN6,2541,5321,5284,3068,4862,2987,3737,3012,5710,...,2750,5196,6130,7446,2285,5576,8742,2716,1262,3413
1,TNMD,0,1,7,0,0,0,1,1,3,...,2,3,9,0,1,1,1,0,1,0
2,DPM1,537,1144,1403,705,1272,405,703,778,1044,...,1375,1862,2470,1265,1105,1091,2211,820,1670,2200
3,SCYL3,414,510,704,288,720,164,543,334,724,...,344,2019,1119,942,585,1077,1166,687,800,406
4,C1orf112,123,146,119,97,107,57,130,91,322,...,134,539,217,396,119,1003,480,136,411,248


In [21]:
lihc_design_matrix.shape

(421, 1)

Create object

In [23]:
# dds = deseq.DESeqDataSetFromMatrix(countData=r_lihc_count_df, colData=r_lihc_label_df, design=design_formula)
deseq2_exp = py_DESeq2(
    count_matrix=lihc_count_matrix,
    design_matrix=lihc_design_matrix,
    design_formula='~ class_label',
    gene_column='gene_symbol'
)

421 | 421


In [24]:
deseq2_exp.run_deseq()

/home/ddomingofernandez/python/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: estimating size factors

  warnings.warn(x, RRuntimeWarning)
/home/ddomingofernandez/python/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: estimating dispersions

  warnings.warn(x, RRuntimeWarning)
/home/ddomingofernandez/python/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: gene-wise dispersion estimates

  warnings.warn(x, RRuntimeWarning)
/home/ddomingofernandez/python/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: mean-dispersion relationship

  warnings.warn(x, RRuntimeWarning)
/home/ddomingofernandez/python/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: final dispersion estimates

  warnings.warn(x, RRuntimeWarning)
/home/ddomingofernandez/python/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: fitting model and testing



In [28]:
results = deseq2_exp.get_deseq_result()

In [29]:
results.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_symbol
0,5000.171755,-0.190154,0.115002,-1.653482,9.823276e-02,1.492199e-01,TSPAN6
1,3.393527,-0.987750,0.364615,-2.709024,6.748146e-03,1.374545e-02,TNMD
2,1212.238480,0.021000,0.066639,0.315137,7.526578e-01,8.072538e-01,DPM1
3,663.394629,0.064535,0.080207,0.804604,4.210483e-01,5.121938e-01,SCYL3
4,305.629500,0.882955,0.153050,5.769042,7.972325e-09,4.282953e-08,C1orf112


In [ ]:
results.to_csv('results.csv')